[62 序列到序列学习](https://www.bilibili.com/video/BV16g411L7FG/?spm_id_from=trigger_reload&vd_source=1d3a7b81d826789081d8b6870d4fff8e)
- seq2seq

典型应用场景机器翻译，一个句子到另一个句子，允许不同长度

<img src="picture/屏幕截图 2022-06-16 003407.png">

seq2seq采用一个rnn作为编码器，一个rnn作为解码器

<img src="picture/屏幕截图 2022-06-16 004107.png">

解码器训练好的隐藏层参数，将继续传递给编码器的隐藏层，作为其初试状态
- 具体实现看代码

<img src="picture/屏幕截图 2022-06-16 005606.png">

In [13]:
import collections
import math
import torch
from d2l import torch as d2l
from torch import nn

In [14]:

class Encoder(nn.Module):
    """The base encoder interface for the encoder-decoder architecture."""
    def __init__(self, **kwargs):
        super(Encoder, self).__init__(**kwargs)

    def forward(self, X, *args):
        raise NotImplementedError

这里说一下概念：编码解码器的原理是，先作为一个整体去训练，训练完后，编码解码的中间层作为输出！这个观点很重要
- 如果是编码器训练完后，再交给解码器训练，那么隐藏层根本没有检验的东西，无法训练
- 如果是编码器解码器统一训练，以实际结果为准，那么本身又等效于一个大的网络了
- 以上两种理解都是不对的

In [15]:
class Seq2SeqEncoder(Encoder):
    """用于序列到序列学习的循环神经网络编码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size) # 输入是vocab，输出编码器size
        # GRU模型，这里其实可以选择多种模型
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers,
                          dropout=dropout)
    
    # encoding是没有输出层的，不需输出最终结果
    def forward(self, X, *args):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X)
        # 在循环神经网络模型中，第一个轴对应于时间步
        X = X.permute(1, 0, 2)
        # 如果未提及状态，则默认为0
        output, state = self.rnn(X)
        # output的形状:(num_steps,batch_size,num_hiddens)
        # state[0]的形状:(num_layers,batch_size,num_hiddens)
        return output, state

torch.Size([1, 2, 3])
torch.Size([1, 2, 3])
torch.Size([1, 3, 2])
torch.Size([2, 1, 3])
torch.Size([3, 2, 1])
